In [1]:
import numpy as np
import random
import pickle
import os
import re
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from colorama import Fore, Back, Style
np.set_printoptions(formatter={'float': '{:.5f}'.format})

In [2]:
class AlakGame:
    def __init__(self, board=None, allow_suicide_moves=False):
        if board is None:
            self.board = "x"*5 + 4*"_" + "o"*5
        else:
            self.board = board
        self.allow_suicide_moves = allow_suicide_moves
        self.X = [] #used for training with random moves
        self.y = []
        self.X_trained = [] #used for training with the trained model
        self.y_trained = []
        self.model = None

    # converting the board list to string
    def board_list_to_str(self, board):
        return ''.join(board)
    
    # converting the board string to list
    def board_str_to_list(self, board):
        return list(board)
    
    # printing the board
    def print_board(self):
        print("Board: ", end='')
        print(self.board)
        print("       0123456789abcd")

    # generate the legal moves
    def generate_legal_moves(self, side, allow_suicide=False):
        legal_moves = []

        for i, piece in enumerate(self.board):
            if piece == side:
                # Check for empty spaces to move the piece
                for j, target in enumerate(self.board):
                    if target == '_':
                        move = (i, j)
                        if allow_suicide or not self.is_suicide_move(side, move):
                            legal_moves.append(move)

        return legal_moves
    
    
    # identify if a move is a suicide move
    def is_suicide_move(self, side, move):
        start, end = move
        board = self.board_str_to_list(self.board)
        board[end] = side
        board[start] = '_'
        piece, board = self.kill_pieces(side, self.board_list_to_str(board))
        return piece == side

    # Given the start and end points, make the move and update the baord
    def make_move(self, side, start, end):
        # Validate the move
        board_dic = {'0': 0,'1': 1, '2': 2, '3':3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
                     '10': 10, '11': 11, '12': 12, '13':13,
                     'a': 10, 'b': 11, 'c': 12, 'd': 13,
                      0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10:10, 11:11, 12:12, 13:13}
        board = self.board_str_to_list(self.board)
        if type(start) == str or type(end) == str:
            if start not in board_dic or end not in board_dic:
                print(Fore.RED + "illegal move, please try again.")
                print("valid i and j options are:", [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'a', 'b', 'c', 'd'])
                print(Style.RESET_ALL)
                return
            else:
                start = board_dic[start]
                end = board_dic[end]
                                   
        if board[start] != side or board[end] != '_' or board[start] == board[end]:
            print(Fore.RED + "illegal move, please try again.")
            print("Valid moves are:", self.generate_legal_moves(side))
            print(Style.RESET_ALL)
            return
            
        if self.allow_suicide_moves:         
            try:
                assert not self.is_suicide_move(side, (start,end))
            except:
                print(Back.RED +"Suicide move!")
                print(Style.RESET_ALL)
        elif not self.allow_suicide_moves and self.is_suicide_move(side, (start,end)):
            raise ValueError("Illegal move! Suicide move!")

        # Move the piece
        board[end] = side
        board[start] = '_'
        self.board = self.board_list_to_str(board)
        
    # given a side and a board, check if the board includes any kills. If so, kill the pieces
    def kill_pieces(self, side, board):
        opponent = 'o' if side == 'x' else 'x'
        pattern_kill = f'{opponent}+'
        regex_kill = re.compile(pattern_kill)
        killed_piece = None
        pattern_suicide = f'{side}+'
        regex_suicide = re.compile(pattern_suicide)

        # check to see if the move results in any kills
        sandwiched_pieces_kill = [f'{side}{match.group()}{side}' for match in regex_kill.finditer(board)]

        for sandwich in sandwiched_pieces_kill:
            if sandwich in board:
                killed_piece = opponent
            board = board.replace(sandwich, side + '_' * (len(sandwich) - 2) + side)

        # check to see if the move is a suicide move
        sandwiched_pieces_suicide = [f'{opponent}{match.group()}{opponent}' for match in regex_suicide.finditer(board)]

        for sandwich in sandwiched_pieces_suicide:
            if sandwich in board:
                killed_piece = side
            board = board.replace(sandwich, opponent + '_' * (len(sandwich) - 2) + opponent)

        return (killed_piece, board)

    
    # check to see if the current state of the board indicates a game over
    def is_game_over(self):
        # checks if the game is over based on the number of pieces remaining for each side
        x_count = self.board.count('x')
        o_count = self.board.count('o')

        if x_count <= 1 or o_count <= 1:
            return True
        return False

    
    # returns the winner of the game based on the remaining pieces on the board
    def winner(self):
        x_count = self.board.count('x')
        o_count = self.board.count('o')

        if x_count <= 1:
            return 'o'
        if o_count <= 1:
            return 'x'
        return None

    
    # resets the board to the initial state    
    def reset_board(self):
        self.board = "x"*5 + 4*"_" + "o"*5

        
    # play a game using random moves or generate the moves using a trained model
    def play_a_game(self, trained = False):
        states = []
        side = 'x'  # 'x' side always starts the game

        while not self.is_game_over():
            if len(states) > 25:
                states = []
                side = 'x'  # 'x' side always starts the game
                counter = 0
                self.reset_board()
            if trained and random.random() > 0.2:
                start, end = self.predict_best_move(self.model, side) # predict the best move 
            else:
                legal_moves = self.generate_legal_moves(side) # make a random move
                start, end = random.choice(legal_moves)
            self.make_move(side, start, end)
            _, self.board = self.kill_pieces(side, self.board)
            states.append(self.board)

            # Switch sides
            side = 'o' if side == 'x' else 'x'
            
        winner = self.winner()
        if winner == "x":
            states.append(self.board)
        self.reset_board()

        return (winner, states)
 

    # convert the game to the format of data to be fed to our model
    def convert_game_to_data(self, game):
        winner, lst = game
        # Stack every two strings in the list using numpy.vstack()
        x_arr = np.vstack([np.array(list(lst[i] + lst[i+1])) for i in range(0, len(lst), 2)])

        # Convert the array elements to integers
        x_arr = np.where(x_arr == 'x', -1, x_arr)
        x_arr = np.where(x_arr == '_', 0, x_arr)
        x_arr = np.where(x_arr == 'o', 1, x_arr)
        x_arr = x_arr.astype(int)
        
        if winner == 'x':
            y_arr = np.full(len(x_arr), 0)
        else:
            y_arr = np.full(len(x_arr), 1)
            
        return (x_arr, y_arr)
            
     
    # generate a set of games to be fed to our model
    def generate_games(self, number, trained=False, brute=False):
        all_states = []
        for i in range(number):
            if(i%1000 == 0):
                print(i)
            if brute: 
                x_arr, y_arr = self.convert_game_to_data(self.generate_brute_force_2sides(self.model))
            else:
                x_arr, y_arr = self.convert_game_to_data(self.play_a_game(trained))
            if trained:
                self.X_trained.append(x_arr)
                self.y_trained.append(y_arr)
            else:
                self.X.append(x_arr)
                self.y.append(y_arr)

                
    # train the model using the generated data
    def train_random_games(self, layers, trained = False):
        if trained:
            X = np.vstack(self.X_trained)
            y = np.hstack(self.y_trained)
        else:   
            X = np.vstack(self.X)
            y = np.hstack(self.y)
        # Split data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        clf = MLPClassifier(hidden_layer_sizes=(layers,), activation='tanh', alpha=0.00001, random_state=42)
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        self.model = clf

        print("Validation accuracy:", accuracy)
            

    # A function that tests to see if the code does the moves and captures correctly
    def test_kill_pieces(self, verbose=False):
        '''
        The tests include, for both sides:
        - simple kill
        - double kill 
        - double kill that involves more than one piece 
        - suicide moves.
        '''
        board_list = ['xoxoxx________',
                      'xooxooxx______',
                      '__xoo__oxo____',
                      '___xoxxxoo____',
                      '___x_oxxoo____',
                      '___xoxx___x___']
        board_expect = ['x_x_xx________',
                        'x__x__xx______',
                        '__xoo__o_o____', 
                        '___xo___oo____', 
                        '___x_o__oo____',
                        '___x_xx___x___']
        off_side = ['x', 'x', 'x', 'o', 'o', 'o']
        fail_num = 0
        print("Testing test_kill_pieces after move from offensive side:")
        for i, board in enumerate(board_list):
            print("\nAfter {:s} move:".format(off_side[i]))
            
            # set board to be the way I want
            self.__init__(board)
            if verbose:
                print("Test {:d}".format(i))
                print('Board: ', self.board)
            _,self.board = self.kill_pieces(off_side[i], self.board)
            try:
                assert self.board == board_expect[i]
            except:
                print('i', i)
                fail_num += 1
        print('{:d} out of {:d} tests passed'.format(len(board_list) - fail_num, len(board_list)))
        if fail_num > 0:
            print(Fore.RED + '{:d} out of {:d} tests failed'.format(fail_num, len(board_list)))
            
        self.reset_board()

        
    # gets a board before and after the move and converts them to numerical representation for prediction
    def convert_board_to_num(self, side, board1, board2):
        board1 = self.board_str_to_list(board1)
        board2 = self.board_str_to_list(board2)
   
        if side == 'x':
            # Reverse the list
            board1 = board1[::-1]
            board2 = board2[::-1]

            # swap "x" and "o"
            board1 = [s.replace("x", "temp").replace("o", "x").replace("temp", "o") for s in board1]
            board2 = [s.replace("x", "temp").replace("o", "x").replace("temp", "o") for s in board2]
            
        # Convert board string to numerical representation for prediction
        temp_board_num = np.array([board1+board2])
        temp_board_num = np.where(temp_board_num == 'x', -1, temp_board_num)
        temp_board_num = np.where(temp_board_num == '_', 0, temp_board_num)
        temp_board_num = np.where(temp_board_num == 'o', 1, temp_board_num)
       
        temp_board_num = temp_board_num.astype(int)
        return temp_board_num
        
        
    # gets the side and the model and predict the best move for the side
    def predict_best_move(self, side, clf):
        board1 = self.board
        best_move = None
        best_probability = -1
        legal_moves = self.generate_legal_moves(side)
        
        if legal_moves == []:
            return (None, None)

        for move in legal_moves:
            # Simulate the move
            start, end = move
            temp_board = self.board_str_to_list(self.board)
            temp_board[end] = side
            temp_board[start] = '_'
            temp_board_num = self.convert_board_to_num(side, board1, self.board_str_to_list(temp_board))

            # Get the predicted probability of winning
            predicted_prob = clf.predict_proba(temp_board_num)[0][1] if side == 'o' else clf.predict_proba(temp_board_num)[0][0]

            if predicted_prob > best_probability:
                best_probability = predicted_prob
                best_move = move
                
        return best_move

    
    # testing the trained model against a random player
    def test_against_random(self, tst_count):
        winner_lst = []
        games = []
        for i in range(tst_count):
            states = []
            side = 'x'  # 'x' side always starts the game

            while not self.is_game_over():
                if side == 'o':
                    start, end = self.predict_best_move(side, self.model) # predict the best move 
                else:
                    legal_moves = self.generate_legal_moves(side, allow_suicide=True) # make a random move
                    start, end = random.choice(legal_moves)
                self.make_move(side, start, end)
                _, self.board = self.kill_pieces(side, game.board)

                states.append(self.board)

                # Switch sides
                side = 'o' if side == 'x' else 'x'
            games.append(states)
            winner_lst.append(self.winner())
            self.reset_board()
        print( "percentage of winning against a random player over 100 games:", winner_lst.count('o')/len(winner_lst)*100, "%")

    
    # print if the killed piece resulted in a gain or loss or there is no kill
    def print_gain(self, side, kill):
        opp = 'o' if side == 'x' else 'x'
        if (kill == opp):
            print(Fore.GREEN + 'gain: 1')
        elif (kill == side):
            print(Fore.RED + 'gain: -1')
        else:
            print(Fore.YELLOW + 'gain: 0')
        print(Style.RESET_ALL)
            
    
    # play interactively for the tournament               
    def interactive_play(self):
        board_dic = {'0': 0,'1': 1, '2': 2, '3':3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
             '10': 10, '11': 11, '12': 12, '13':13, 'a': 'a', 'b': 'b', 'c': 'c', 'd': 'd',
             0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9,
             10: 'a', 11: 'b', 12: 'c', 13: 'd'}
        i = 0
        print("____________Welcome to AlAK Game!____________")
        side = input('Please pick your side: ')
        print("****Starting Game****")
        while not self.is_game_over():
            print("round: ", i)
            print("Please enter your move:")
            self.print_board()
            if side == 'x':
                if self.board.count('o')<=1 or self.board.count('x')<=1:
                    break
                # x makes a move
                temp_board = self.board
                while temp_board == self.board:
                    start = input('Select piece(i): ')
                    end = input('Select position(j): ')
                    self.make_move(side, start, end)
                print("x: {} ==> {}".format(board_dic[start], board_dic[end]))
                kill, self.board = self.kill_pieces(side, game.board)
                self.print_board()
                self.print_gain(side, kill)
                
                # model (o) makes a move
                if self.board.count('o')<=1 or self.board.count('x')<=1:
                    break
                start, end = self.predict_best_move('o', self.model)
                print("o: {} ==> {}".format(board_dic[start], board_dic[end]))
                self.make_move('o', start, end)
                kill, self.board = self.kill_pieces('o', game.board)
                self.print_board()
                self.print_gain('o', kill)
                print('________________________')
                              
            else:
                if self.board.count('o')<=1 or self.board.count('x')<=1:
                    break
                # model (x) makes a move
                start, end = self.predict_best_move('x', self.model)
                print("x: {} ==> {}".format(board_dic[start], board_dic[end]))
                self.make_move('x', board_dic[start], board_dic[end])
                kill, self.board = self.kill_pieces('x', game.board)
                self.print_board()
                self.print_gain('x', kill)
                
                # o makes a move
                temp_board = self.board
                if self.board.count('o')<=1 or self.board.count('x')<=1:
                    break
                while temp_board == self.board:
                    start = input('Select piece(i): ')
                    end = input('Select position(j): ')
                    self.make_move(side, start, end)
                print("o: {} ==> {}".format(board_dic[start], board_dic[end]))
                kill, self.board = self.kill_pieces(side, game.board)
                self.print_board()
                self.print_gain(side, kill)
                print('________________________')
                
            i += 1
        print(Fore.MAGENTA + "｡*ﾟ✲*☆｡*ﾟ✲*---- (｡•̀ᴗ-)✧ player", self.winner(), "won! ---- ｡*ﾟ✲*☆｡*ﾟ✲*")
        print(Style.RESET_ALL)

        self.reset_board()
        
    
    # return the number of killed peices
    def kill_num(self, side, board1, board2):
        opp = 'o' if side == 'x' else 'x'
        if side == 'x':
            return board1.count('o') - board2.count('o')
        else:
            return board1.count('x') - board2.count('x')
        

    # generate a game with one side using a trained model to make moves and the other make moves based on 
    # the greatest number of kills in each round using brute force
    def generate_brute_force(self, clf):
        states = []
        side = 'x'  # 'x' side always starts the game

        while not self.is_game_over():
            if len(states) > 25:
                states = []
                side = 'x'  # 'x' side always starts the game
                self.reset_board()

            legal_moves = self.generate_legal_moves(side)

            if side == 'o':
                best_move = None
                max_kills = -1
                best_probability = -1

                for move in legal_moves:
                    start, end = move

                    temp_board = self.board_str_to_list(self.board)
                    temp_board[end] = side
                    temp_board[start] = '_'
                    killed_piece, temp_board_str = self.kill_pieces(side, self.board_list_to_str(temp_board))

                    # Count the number of pieces killed
                    kills = self.kill_num(side, self.board, temp_board_str)

                    # If more pieces are killed or equal and probability is higher, update the best move
                    if kills >= max_kills:
                        max_kills = kills
                        best_move = move

            else:
                # 70% of times, choose the move with the highest predicted probability of winning
                if random.random() < 0.5:
                    start, end = self.predict_best_move(side, self.model) # predict the best move 
                    best_move = (start, end)

                else:
                    # 30% of times, choose a random move
                    best_move = random.choice(legal_moves)

            start, end = best_move
            self.make_move(side, start, end)
            _, self.board = self.kill_pieces(side, self.board)

            states.append(self.board)

            # Switch sides
            side = 'o' if side == 'x' else 'x'

        winner = self.winner()
        if winner == "x":
            states.append(self.board)
        self.reset_board()

        return (winner, states)

    
    # generate a game with both sides making moves based on the greatest number of kills in each round using brute force
    def generate_brute_force_2sides(self, clf):

        states = []
        side = 'x'  # 'x' side always starts the game
        best_moves = []

        while not self.is_game_over():
            if len(states) > 25:
                states = []
                side = 'x'  # 'x' side always starts the game
                self.reset_board()
                best_moves = []

            legal_moves = self.generate_legal_moves(side)

            best_move = None
            max_kills = -1
            best_probability = -1

            for move in legal_moves:
                start, end = move

                # Simulate the move and kill pieces
                temp_board = self.board_str_to_list(self.board)
                temp_board[end] = side
                temp_board[start] = '_'

                killed_piece, temp_board_str = self.kill_pieces(side, self.board_list_to_str(temp_board))

                # Count the number of pieces killed
                kills = self.kill_num(side, self.board, temp_board_str)

                # If more pieces are killed or equal and probability is higher, update the best move
                if kills > max_kills:
                    max_kills = kills
                    best_moves = [move]
                elif kills == max_kills:
                    best_moves.append(move)
            
            best_move = random.choice(best_moves)

            start, end = best_move
            self.make_move(side, start, end)
            _, self.board = self.kill_pieces(side, self.board)

            states.append(self.board)

            # Switch sides
            side = 'o' if side == 'x' else 'x'

        winner = self.winner()
        if winner == "x":
            states.append(self.board)
        self.reset_board()

        return (winner, states)




In [3]:
# Load the model from the pickle file
with open('model_brute_3000000_10.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [4]:
game = AlakGame(allow_suicide_moves=True)
game.model = loaded_model

In [12]:
game.test_against_random(100)

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide move!

Suicide mo

In [17]:
game.interactive_play()

____________Welcome to AlAK Game!____________
Please pick your side: o
****Starting Game****
round:  0
Please enter your move:
Board: xxxxx____ooooo
       0123456789abcd
x: 4 ==> 8
Board: xxxx____xooooo
       0123456789abcd
gain: 0

Select piece(i): d
Select position(j): 5
o: d ==> 5
Board: xxxx_o__xoooo_
       0123456789abcd
gain: 0

________________________
round:  1
Please enter your move:
Board: xxxx_o__xoooo_
       0123456789abcd
x: 8 ==> 6
Board: xxxx_ox__oooo_
       0123456789abcd
gain: 0

Select piece(i): c
Select position(j): 8
o: c ==> 8
Board: xxxx_ox_oooo__
       0123456789abcd
gain: 0

________________________
round:  2
Please enter your move:
Board: xxxx_ox_oooo__
       0123456789abcd
x: 3 ==> 4
Board: xxx_x_x_oooo__
       0123456789abcd
gain: 1

Select piece(i): 8
Select position(j): c
o: 8 ==> c
Board: xxx_x_x__oooo_
       0123456789abcd
gain: 0

________________________
round:  3
Please enter your move:
Board: xxx_x_x__oooo_
       0123456789abcd
x: 6 ==> 8
Bo